In [ ]:
import psycopg2
from dotenv import load_dotenv   #for python-dotenv method
load_dotenv()                    #for python-dotenv method
import os 
import pandas as pd

In [ ]:
db_host = os.environ.get('HOST')
db_database = os.environ.get('DATABASE')
db_user = os.environ.get('USER')
db_port = os.environ.get('PORT')
db_password = os.environ.get('PASSWORD')
db_uri = os.environ.get('URI')

In [ ]:
def connect_postgres():
    conn = psycopg2.connect(
    host=db_host,
    database=db_database,
    user=db_user,
    password=db_password)
    return conn

In [ ]:
conn = connect_postgres()

In [ ]:
sql = """
select v.companycode, v.companyname, 
    v.nemonico, v.sectorcode,
    v.sectordescription, v."rpjCode", 
    v.website, v."esActDescription",
    v.coin,v.anho,
    sum(valor)
from
(
    select a.companycode, a.companyname, 
        a.nemonico, a.sectorcode,
        a.sectordescription, a."rpjCode", 
        a.website, a."esActDescription",
        s.coin,
        cast(s.benefitvalue as float) as valor,
        s.datecut,
        EXTRACT(YEAR FROM to_date(s.datecut, 'YYYY-MM-DD')) as anho,
        EXTRACT(MONTH FROM to_date(s.datecut, 'YYYY-MM-DD')) as mes
    from 
        public.companystock a,
        public.stockcompanyvalue s
    where a."rpjCode" is not null and 
     s.codigo = a.companycode and
s.benefittype = 'DE'
and s.benefitvalue != 'None'
order by a.sectorcode 
) v
group by v.companycode, v.companyname, 
     v.nemonico, v.sectorcode,
       v.sectordescription, v."rpjCode", 
   v.website, v."esActDescription",
  v.coin,v.anho
order by v.sectorcode,v.nemonico,v.coin,v.anho
"""

In [ ]:
df = pd.read_sql_query(sql, conn)

In [ ]:
df.head()

In [ ]:
#df[df.nemonico == "BAP"]["anho"].to_list()
df[df.nemonico == "BAP"]["sum"].to_list()

In [ ]:
#!pip install dash-renderer==0.10.0
#!pip install dash-html-components==0.7.0
#!pip install dash-core-components==0.12.6
#!pip install plotly --upgrade
#!pip install jupyter-dash

In [ ]:
arr_sum = df[df.nemonico == "BAP"]["sum"].to_list()
arr_anho = df[df.nemonico == "BAP"]["anho"].to_list()

In [ ]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
# Load Data
df = px.data.tips()
# Build App
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("JupyterDash Demo"),
    dcc.Graph(id='graph'),
    html.Label([
        "colorscale",
        dcc.Dropdown(
            id='colorscale-dropdown', clearable=False,
            value='plasma', options=[
                {'label': c, 'value': c}
                for c in px.colors.named_colorscales()
            ])
    ]),
])

# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    [Input("colorscale-dropdown", "value")]
)
def update_figure(colorscale):
    return px.scatter(
        df, x="total_bill", y="tip", color="size",
        color_continuous_scale=colorscale,
        render_mode="webgl", title="Tips"
    )
# Run app and display result inline in the notebook
app.run_server(mode='inline')

In [2]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from jupyter_dash import JupyterDash
 
#app = dash.Dash()
app = JupyterDash(__name__)
 
app.layout = html.Div(children=[
  html.H1(children='Tutoriales Dash en AprenderPython.com'),
  dcc.Graph(id='ejemplo',
    figure={
      'data': [
        {'x': [1, 2, 3, 4], 'y': [1, 8, 3, 1], 'type': 'line', 'name': 'Bicicletas'},
        {'x': [1, 2, 3, 4], 'y': [5, 2, 8, 8], 'type': 'bar', 'name': 'Bicicletas electricas'},
        ],
      'layout': {
      'title': 'Ejemplo básico en Dash 3'
        }
      })
  ])
 
app.run_server(mode='jupyterlab',debug=True)

Exception in thread Thread-12:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\ProgramData\Anaconda3\lib\site-packages\retrying.py", line 49, in wrapped_f
    return Retrying(*dargs, **dkw).call(f, *args, **kw)
  File "C:\ProgramData\Anaconda3\lib\site-packages\retrying.py", line 212, in call
    raise attempt.get()
  File "C:\ProgramData\Anaconda3\lib\site-packages\retrying.py", line 247, in get
    six.reraise(self.value[0], self.value[1], self.value[2])
  File "C:\ProgramData\Anaconda3\lib\site-packages\six.py", line 703, in reraise
    raise value
  File "C:\ProgramData\Anaconda3\lib\site-packages\retrying.py", line 200, in call
    attempt = Attempt(fn(*args, **kwargs), attempt_number, False)
  File "C:\ProgramData\Anaconda3\lib\site-packages\jupyter_dash\jupyter_app.py", li

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))